In [12]:
import pandas as pd
from global_variables.global_variables import filename_proc, sleep_activities, start_date
from supporting_func.data_summaries import strip_activity, combine_sleep, split_data
from datetime import datetime, timedelta

In [13]:
#download just the sleep data
df = pd.read_csv(filename_proc)
df = df[sleep_activities]
time_frame = 30
data_dict = split_data(df,time_frame)
df_train = data_dict[0]
df_test = data_dict[1]



In [14]:
#create pd start sleep, stop sleep, time slept, date
sleep_summary_train = strip_activity('Sleep', df_train)
print(sleep_summary_train)


   Activity               start                 end  Duration
0     Sleep 2012-01-28 01:08:00 2012-01-28 01:30:00  0.366667
1     Sleep 2012-01-28 02:16:00 2012-01-28 05:01:00  2.750000
2     Sleep 2012-01-28 05:05:00 2012-01-28 07:16:00  2.183333
3     Sleep 2012-01-28 21:04:00 2012-01-29 03:55:00  6.850000
4     Sleep 2012-01-29 03:58:00 2012-01-29 06:04:00  2.100000
5     Sleep 2012-01-29 06:24:00 2012-01-29 07:38:00  1.233333
6     Sleep 2012-01-29 21:25:00 2012-01-30 03:19:00  5.900000
7     Sleep 2012-01-30 03:23:00 2012-01-30 07:06:00  3.716667
8     Sleep 2012-01-30 21:22:00 2012-01-31 01:46:00  4.400000
9     Sleep 2012-01-31 01:55:00 2012-01-31 04:12:00  2.283333
10    Sleep 2012-01-31 04:27:00 2012-01-31 07:00:00  2.550000
11    Sleep 2012-01-31 22:23:00 2012-02-01 02:42:00  4.316667
12    Sleep 2012-02-01 02:48:00 2012-02-01 05:45:00  2.950000
13    Sleep 2012-02-01 05:49:00 2012-02-01 06:53:00  1.066667
14    Sleep 2012-02-01 21:26:00 2012-02-02 01:41:00  4.250000
15    Sl

In [15]:
end_date_train = start_date + timedelta(days=time_frame)
daily_sleep_sum_train = combine_sleep(sleep_summary_train,start_date, end_date_train)
print(daily_sleep_sum_train)

                  Date  total_sleep first_sleep last_sleep  sleep_count
0  2012-01-28 12:00:00        10.18    21:04:00   07:38:00            3
1  2012-01-29 12:00:00         9.62    21:25:00   07:06:00            2
2  2012-01-30 12:00:00         9.23    21:22:00   07:00:00            3
3  2012-01-31 12:00:00         8.33    22:23:00   06:53:00            3
4  2012-02-01 12:00:00         9.98    21:26:00   07:35:00            4
5  2012-02-02 12:00:00         8.72    22:05:00   07:01:00            3
6  2012-02-03 12:00:00         8.25    22:54:00   07:15:00            3
7  2012-02-04 12:00:00         8.50    22:26:00   07:00:00            2
8  2012-02-05 12:00:00         9.18    21:48:00   07:12:00            4
9  2012-02-06 12:00:00         7.35    23:14:00   06:40:00            2
10 2012-02-07 12:00:00         8.63    22:01:00   06:50:00            3
11 2012-02-08 12:00:00         9.42    21:48:00   07:27:00            4
12 2012-02-09 12:00:00         8.78    23:04:00   08:01:00      

In [16]:
#find the distributions for time slept, average bed time, average wake time
average_sleep = daily_sleep_sum_train['total_sleep'].mean()
median_sleep = daily_sleep_sum_train['total_sleep'].mean()

std_sleep = daily_sleep_sum_train['total_sleep'].std()
print(average_sleep)
print(std_sleep)

8.499
1.913053796312641


In [17]:
sleep_summary_test = strip_activity('Sleep', df_test)
start_date_test = end_date_train
end_date_test = datetime.strptime(df['Date'][df_test.index.tolist()[-1]], '%Y-%m-%d %H:%M:%S')

In [18]:
daily_sleep_sum_test = combine_sleep(sleep_summary_test,start_date_test, end_date_test)
print(daily_sleep_sum_test)

                  Date  total_sleep first_sleep last_sleep  sleep_count
0  2012-02-27 12:00:00         7.48    23:19:00   06:54:00            2
1  2012-02-28 12:00:00         8.05    22:45:00   06:59:00            3
2  2012-02-29 12:00:00         9.72    14:30:00   06:45:00            4
3  2012-03-01 12:00:00         6.78    00:16:00   07:06:00            2
4  2012-03-02 12:00:00         8.53    23:07:00   07:49:00            3
5  2012-03-03 12:00:00         9.27    21:30:00   06:59:00            3
6  2012-03-04 12:00:00         9.87    20:59:00   07:07:00            4
7  2012-03-05 12:00:00         7.47    22:43:00   06:23:00            2
8  2012-03-06 12:00:00         9.00    23:30:00   08:40:00            3
9  2012-03-07 12:00:00         9.38    22:03:00   07:39:00            3
10 2012-03-08 12:00:00         9.88    00:19:00   11:54:00            5
11 2012-03-09 12:00:00         9.07    23:00:00   08:14:00            3
12 2012-03-10 12:00:00        10.38    22:14:00   08:44:00      